### Dyer-Lashof-Cohen basis

In [ ]:
from itertools import combinations, product, combinations_with_replacement, permutations, combinations, product, chain, tee
from sympy.utilities.iterables import multiset_permutations
from clesto import *

def symmetric_orbit(surj):
    translation = {(1,2): (1,2,3), (1,3): (1,3,2), (2,1): (2,1,3),
                   (2,3): (3,1,2), (3,1): (2,3,1), (3,2): (3,2,1)}
    rep = Surjection_element(torsion=3)
    for k, v in surj.items():
        perm = SymmetricModule_element({translation[k[:2]]: 1}, torsion=surj.torsion)
        new = perm * Surjection_element({k:v}, torsion=surj.torsion)
        rep += new
    return rep

def sign_symmetric_orbit(surj):
    translation = {(1,2): (1,2,3), (1,3): (1,3,2), (2,1): (2,1,3),
                   (2,3): (3,1,2), (3,1): (2,3,1), (3,2): (3,2,1)}
    sign = {(1,2): 1, (1,3): -1, (2,1): -1,
            (2,3): 1, (3,1): 1, (3,2): -1}
    rep = Surjection_element(torsion=3)
    for k, v in surj.items():
        perm = SymmetricModule_element({translation[k[:2]]: sign[k[:2]]}, torsion=surj.torsion)
        new = perm * Surjection_element({k:v}, torsion=surj.torsion)
        rep += new
    return rep

# def sign_permutation(permutation):
#     perm = [i - 1 for i in permutation]
#     visited = [False] * len(perm)
#     parity = 0
#     for i in perm:
#         visited[i] = True
#         cycle_length = 0
#         while not visited[perm[i]]:
#             visited[perm[i]] = True
#             i = perm[i]
#             cycle_length += 1
#         parity += cycle_length % 2

#     return (-1)**parity 

def symmetric_orbit_barratt_eccles(be, sign_representation=False):
    inverses = {(1,2,3): (1,2,3), (2,3,1): (3,1,2), (3,1,2): (2,3,1), 
                (1,3,2): (1,3,2), (3,2,1): (3,2,1), (2,1,3): (2,1,3)}
    signs = {(1,2,3): 1, (2,3,1): 1, (3,1,2): 1, 
             (1,3,2): -1, (3,2,1): -1, (2,1,3): -1}
    
    answer = BarrattEccles_element(torsion=be.torsion)
    for k, v in be.items():
        inverse = k[-1]
        perm = SymmetricModule_element({inverse: 1}, torsion=be.torsion)
        new = perm * BarrattEccles_element({k: v}, torsion=be.torsion)
        if sign_representation:
            new = signs[k[-1]] * new
        answer += new
        
    return answer

def extra_reduce(surj, r=3):
    return Surjection_element({k: v for k, v in surj.items() if set(k) == set(range(1, r+1))})

def cyclic_orbit(surj):
    pass

def get_symmetric_basis():
    basis = set()
    for x in combinations_with_replacement((1,2,3), 5):
        for y in permutations(x):
            if set(y) == {1,2,3} and Surjection_element({y:1}):
                basis.update(symmetric_orbit(Surjection_element({y:1}, torsion=3)))
    return basis

def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

def get_basis(r, n, torsion='free'):
    basis = []
    surjections = (chain.from_iterable(multiset_permutations(y) 
        for y in combinations_with_replacement(range(1, r+1), r+n)))
    for s in surjections:
        if all([i != j for i, j in pairwise(s)]) and set(s) == set(range(1, r+1)):
#             yield Surjection_element({tuple(s):1}, torsion=torsion)
            yield tuple(s)
        
def phi(i, j, permutations):
    '''the restriction to a pair of integers a la Berger, i.e pull back to the arity 2 part'''
    def _phi(i, j, permutation):
        try:
            a, b = permutation.index(i), permutation.index(j)

        except ValueError:
            print(i, j, permutation)

        if a < b:
            return (1, 2)
        else:
            return (2, 1)
        
    return tuple(_phi(i, j, p) for p in permutations)

def is_nondegenerate(spx):
    return all(spx[i] != spx[i + 1] for i in range(len(spx) - 1))

def phi(i, j, barratt_eccles_element):
    
    def _phi(i, j, permutation):
        a, b = permutation.index(i), permutation.index(j)
        if a < b:
            return (1, 2)
        else:
            return (2, 1)
        
    if isinstance(barratt_eccles_element, tuple):
        return tuple(_phi(i, j, p) for p in barratt_eccles_element)
    
    torsion = barratt_eccles_element.torsion
    answer = BarrattEccles_element(torsion=torsion)
    for k, v in barratt_eccles_element.items():
        new_simplex = []
        new_simplex = phi(i, j, k)
        new_summand = BarrattEccles_element({new_simplex: v}, torsion=torsion)
        answer += new_summand

def complexity(barratt_eccles_element):
    "Not finished yet"
    
    def _complexity(spx):
        complexities = [0]
        for i, j in combinations(sorted(spx[0]), 2):
            cpxty = len([p for p, q in pairwise(phi(i, j, spx)) if p != q])
            complexities.append(cpxty)
        return max(complexities)
    
    if isinstance(barratt_eccles_element, tuple):
        return _complexity(barratt_eccles_element)
    
def act (perm1, perm2):
    return tuple(perm1[i-1] for i in perm2)

inverse = {(1, 3, 2): (1, 3, 2), (3, 2, 1): (3, 2, 1), (2, 1, 3): (2, 1, 3),
           (1, 2, 3): (1, 2, 3), (3, 1, 2): (2, 3, 1), (2, 3, 1): (3, 1, 2)}

## Search among all surjections

In [ ]:
def complexity(i, j, s):
    phi = (k for k in s if k == i or k == j)
    return -1 + len([p for p, q in pairwise(phi) if p != q])

def final_occurence(i, j, s):
    phi = tuple(k for k in s if k == i or k == j)
    if phi[-1] == j:
        return (1, 2)
    else:
        return (2, 1)

r = 3 # arity
d = 2 # dimension
c = 1 # complexity i.e. in E_{c+1}

all_surjections = (s for s in product(range(1, r+1), repeat = r+d)
    if all(i != j for i, j in pairwise(s)) 
    and set(s) == set(range(1, r+1)))

good_surjections = tuple(s for s in all_surjections
    if all(complexity(i, j, s) <= c for i, j in combinations(range(1, r+1), 2)))

cells = {tuple(pi) : None for pi in permutations(range(1, r+1))}

for k in cells.keys():
    cells[k] = tuple(s for s in good_surjections if 
                     all(complexity(i, j, s) < c 
                         or final_occurence(i, j, s) == final_occurence(i, j, k)
                         for i, j in combinations(range(1, r+1), 2)))

for k, v in cells.items():
    print(k, ':')
    print(v)
    print('---')

In [ ]:
# surjections shared by different cells

for x, y in product(permutations(range(1, r+1)), repeat=2):
    if x != y:
        print(x, y)
        print(set(cells[x]).intersection(cells[y]))
        print('---')

In [ ]:
# complexity and final occurrence in a surjs of a cell

a = (1,2,3)
for s in cells[a]:
    print(s)
    for i,j in combinations(range(1,4), 2):
        print(complexity(i,j,s), final_occurence(i,j,s))
    print('---')

In [ ]:
x = Surjection_element({(1,2,1,2) :1}, torsion='free')
y = Surjection_element({(1,2,1) :1}, torsion=3)

# as you read it
a = x.compose(x, 2)
b = x.compose(x, 1)
c = x.compose(y, 2)
d = x.compose(y, 1)
e = y.compose(x, 2)
f = y.compose(x, 1)

print(e)

e.orbit(representation=sign)


In [ ]:
# search for coefficients for elements in a cell
# remember to change the representation

identity = (1,2,3)
cycles = []
cell = cells[identity]

for coeffs in product({0,1,2}, repeat=len(cell)):
    eta = Surjection_element(torsion=3)
    for idx, s in enumerate(cell):
        eta += Surjection_element({s: coeffs[idx]}, torsion=3)
        
    if not symmetric_orbit(eta.boundary()) and symmetric_orbit(eta):
        cycles.append(eta)
        
# for cycle in cycles:
#     print(cycle)
#     print("---")

In [ ]:
# distinct cycle orbits

orbits = []

for surj in cycles:
    if not symmetric_orbit(surj) in orbits:
        orbits.append(symmetric_orbit(surj))

print(orbits[1])
(1,2,3,2,3,1) + 2(1,2,3,2,1,3) + 2(1,2,3,2,1,2) + (1,2,1,2,3,2)

In [ ]:
p = ((1,2,1,3,1,3,2), (2,1,3,1,3,2,3), (1,3,1,2,1,2,3), (1,2,3,1,3,2,3), (1,2,1,2,3,2,3), (1,2,3,2,1,2,3), (1,2,3,2,3,1,3), (1,2,1,3,1,2,3))
set(p) == set(cell) 